In [20]:
import numpy as np
import pandas as pd
from CreateGameResultsTable import createGameResultsCSV
import requests
from bs4 import BeautifulSoup, Comment
from io import StringIO

In [21]:
df = pd.read_csv('nhlGameResults.csv.gz', compression='gzip')

In [22]:
df

,Date,Season,Visiting Team,Home Team,Margin,Home Win,Home Back to Back,Visiting Back to Back,S%Diff,SV%Diff,...,SCFDiff,SCADiff,SCF%Diff,HDFDiff,HDADiff,HDF%Diff,HDGFDiff,HDC%Diff,HDGADiff,HDCO%Diff
0,2021-01-13,2021,St. Louis Blues,Colorado Avalanche,-3.0,0,False,False,0.523072,0.274390,...,3.357292,-2.723369,3.303754,3.020588,-2.471422,3.100573,1.065622,-2.035288,-2.025764,-0.742525
1,2021-01-13,2021,Vancouver Canucks,Edmonton Oilers,-2.0,0,False,False,0.784608,0.411585,...,-1.631421,1.071489,-1.443156,-1.470895,1.235711,-1.456777,-1.522317,-0.407058,-1.485560,-3.013776
2,2021-01-13,2021,Pittsburgh Penguins,Philadelphia Flyers,3.0,1,False,False,-1.307679,-3.429875,...,1.631421,-1.544730,1.848671,1.864885,-1.961020,2.254067,0.913390,-1.475584,-1.350509,-0.043678
3,2021-01-13,2021,Chicago Blackhawks,Tampa Bay Lightning,4.0,1,False,False,0.523072,1.097560,...,-1.227859,0.383950,-0.918372,-0.814245,0.483539,-0.757918,0.304463,1.831759,-0.135051,-0.655169
4,2021-01-13,2021,Montreal Canadiens,Toronto Maple Leafs,1.0,1,False,False,2.615359,1.097560,...,0.695500,-0.160723,0.393588,-0.262660,-0.402949,0.137803,1.522317,2.086170,-1.350509,-1.878150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4793,2024-04-16,2024,Seattle Kraken,Winnipeg Jets,1.0,1,False,False,1.249402,1.503226,...,0.115356,0.057768,0.026662,0.140280,0.461536,-0.196015,1.746780,2.068679,-0.243207,-0.598552
4794,2024-04-17,2024,Edmonton Oilers,Arizona Coyotes,3.0,1,False,False,0.832935,-0.250538,...,-2.171405,0.975641,-1.706373,-2.073027,1.335158,-2.033655,-2.390331,-0.954775,0.243207,-0.897828
4795,2024-04-17,2024,St. Louis Blues,Dallas Stars,1.0,1,False,False,1.249402,-1.002151,...,2.442831,-1.906350,2.452911,1.402800,-2.307680,2.352180,1.746780,1.034340,0.486414,2.593726
4796,2024-04-17,2024,Pittsburgh Penguins,New York Islanders,1.0,1,False,False,-0.277645,1.377957,...,-0.814277,0.487820,-0.693214,-0.561120,1.038456,-0.906569,-0.551615,-0.159129,-1.580847,-2.493967


In [23]:
req_url = 'https://www.hockey-reference.com/leagues/NHL_2024.html'
data = requests.get(req_url)
soup = BeautifulSoup(data.content, 'html.parser')
table = soup.select_one('h2:-soup-contains("Team Statistics")').find_next(string=lambda t: isinstance(t, Comment))

soup = BeautifulSoup(str(table), 'html.parser')
table = soup.find('table', attrs={'id': 'stats'})
df = pd.read_html(StringIO(str(table)))[0]

In [27]:
df = df[['Unnamed: 1_level_0', 'Special Teams']]

In [28]:
columnlist = list()

for index in df.columns:
    columnlist.append(index[1])

df = pd.DataFrame(df.values, columns=columnlist)
df = df.rename(columns={'Unnamed: 1_level_1':'Team'})

In [29]:
df

,Team,PP,PPO,PP%,PPA,PPOA,PK%,SH,SHA,PIM/G,oPIM/G
0,New York Rangers*,65,246,26.42,36,232,84.48,8,6,9.4,10.4
1,Dallas Stars*,58,240,24.17,43,239,82.01,12,4,6.8,7.5
2,Carolina Hurricanes*,67,249,26.91,35,258,86.43,11,8,8.5,8.0
3,Florida Panthers*,63,268,23.51,51,291,82.47,8,9,13.6,13.5
4,Boston Bruins*,54,243,22.22,47,268,82.46,7,6,9.5,8.8
5,Vancouver Canucks*,57,254,22.44,53,250,78.8,7,7,9.6,9.8
6,Winnipeg Jets*,45,236,19.07,50,220,77.27,3,5,8.2,8.9
7,Colorado Avalanche*,67,274,24.45,52,262,80.15,7,6,9.0,9.7
8,Edmonton Oilers*,63,241,26.14,52,254,79.53,7,5,9.5,8.5
9,Toronto Maple Leafs*,57,238,23.95,55,238,76.89,6,9,9.9,10.2
